# Demo 2 A - nearest spaceport

In [ ]:
!pip install --quiet "astrapy>=1.0.0" "python-dotenv>=1.0.0"

In [ ]:
import getpass
import os

from dotenv import load_dotenv

from astrapy import DataAPIClient
from astrapy.constants import VectorMetric
from astrapy.ids import ObjectId

## Setup DB

In [ ]:
load_dotenv()

if "ASTRA_DB_APPLICATION_TOKEN" not in os.environ:
    os["ASTRA_DB_APPLICATION_TOKEN"] = getpass.getpass("Please input your Astra DB Token:")

if "ASTRA_DB_API_ENDPOINT" not in os.environ:
    os.environ["ASTRA_DB_API_ENDPOINT"] = input("Please input your Astra DB API Endpoint:")

if "ASTRA_DB_KEYSPACE" not in os.environ:
    _namespace = input("(Optional) Input your Astra DB namespace if desired, or leave blank:")
    if _namespace:
        os.environ["ASTRA_DB_KEYSPACE"] = _namespace

ASTRA_DB_APPLICATION_TOKEN = os.environ["ASTRA_DB_APPLICATION_TOKEN"]
ASTRA_DB_API_ENDPOINT = os.environ["ASTRA_DB_API_ENDPOINT"]
ASTRA_DB_KEYSPACE = os.environ.get("ASTRA_DB_KEYSPACE")

In [ ]:
db = DataAPIClient(ASTRA_DB_APPLICATION_TOKEN).get_database_by_api_endpoint(ASTRA_DB_API_ENDPOINT, namespace=ASTRA_DB_KEYSPACE)

In [ ]:
spaceports_collection = db.create_collection(
    "craftdemo3d_euclidean",
    dimension=3,
    metric=VectorMetric.EUCLIDEAN,
    check_exists=False,
)
spaceports_collection.delete_all()

## Data input

In [ ]:
spaceports = [
    {
        "_id": ObjectId("6642938483ffc52bf749ab6a"),
        "name": "Cygnus X",
        "metadata": {
            "rating": 9.0,
            "restaurant": False,
            "tags": ["killer_robots"],
        },
        "$vector": [100, 10, 70],
    },
    {
        "_id": ObjectId("6642939583ffc52bf749ab70"),
        "name": "Betelgeuse",
        "metadata": {
            "rating": 6.5,
            "restaurant": False,
            "tags": ["garden", "playground"],
        },
        "$vector": [20, 0, 50],
    },
    {
        "_id": ObjectId("6642939d83ffc52bf749ab74"),
        "name": "Orion's Belt",
        "metadata": {
            "rating": 4.5,
            "restaurant": True,
            "tags": ["playground", "lightsabers"],
        },
        "$vector": [40, 80, 30],
    },
]

In [ ]:
im_result = spaceports_collection.insert_many(spaceports)
for inserted_id in im_result.inserted_ids:
    print(f"Inserted: {inserted_id}")

## Find nearest spaceport

In [ ]:
# your galactic coordinates:
my_location = [0, 10, 50]

In [ ]:
result1 = spaceports_collection.find_one(vector=my_location)
if result1:
    print(f"Closest => {result1['name']} ({result1['_id']})")
else:
    print("(no matches)")

In [ ]:
result2 = spaceports_collection.find_one(
    vector=my_location,
    filter={"metadata.restaurant": True},
)

if result2:
    print(f"Closest => {result2['name']} ({result2['_id']})")
else:
    print("(no matches)")

In [ ]:
result3 = spaceports_collection.find_one(
    filter={"metadata.rating": {"$gt": 7.2}},
    vector=my_location,
)
if result3:
    print(f"Closest => {result3['name']} ({result3['_id']})")
else:
    print("(no matches)")

In [ ]:
result4 = spaceports_collection.find_one(
    filter={"metadata.rating": {"$gt": 7.2}, "metadata.restaurant": True},
    vector=my_location,
)
if result4:
    print(f"Closest => {result4['name']} ({result4['_id']})")
else:
    print("(no matches)")